## Introduction: The Lemonade Stand

Welcome to our interactive tutorial where we'll dive into the exciting world of decision-making and strategy through the lens of running a lemonade stand. This journey isn't just about selling lemonade; it's a comprehensive exploration of how to navigate uncertainties, optimize decisions, and adapt strategies in a business context, all wrapped in the fun and accessible scenario of a lemonade stand.

### Why a Lemonade Stand?

A lemonade stand, though simple, encapsulates the essence of business: understanding customer needs, dealing with uncertainties (like the weather), setting prices, and marketing. These elements are not just the core of running a lemonade stand but are fundamental to managing any business, making it a perfect, relatable model for learning.

### What you will learn

Through this tutorial, you will:
- **Understand Uncertainty**: Learn to identify and characterize uncertainties (e.g., weather conditions, lemon costs) and their impact on business decisions.
- **Explore Scenarios**: Dive into scenario analysis to understand how different conditions affect outcomes and how to prepare for them.
- **Optimize Decisions**: Discover how to make the best decisions in light of these uncertainties and scenarios, balancing various objectives like profit, costs, and time investment.
- **Adapt Strategies**: Learn about adaptive policymaking, adjusting your strategy as you gain more information and as conditions change.

### The Simulation Model

Our exploration is centered around a Python simulation model of a lemonade stand. This model will help us understand the dynamics of running the stand under different conditions. It takes into account factors such as:
- Weather (sunny, cloudy, rainy)
- Day type (weekday, weekend)
- Pricing strategies
- Marketing efforts
- Hours of operation
- Uncertain lemon purchase prices
- Special events that might increase demand

We'll use this model to simulate various scenarios, analyze the outcomes, and optimize our lemonade stand's operations.

In [ ]:
# Some imports to get us started
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def simulate_lemonade_stand(weather, day, price, marketing_spend, hours_open, lemon_cost, special_event=False):
    """
    Simulates a day at the lemonade stand, considering various factors including time spent and lemon cost.

    Parameters:
    - weather: 'sunny', 'cloudy', or 'rainy'
    - day: 'weekday' or 'weekend'
    - price: Price per cup of lemonade
    - marketing_spend: Money spent on marketing that day
    - hours_open: Hours the lemonade stand is open
    - lemon_cost: Cost per lemon (uncertain variable)
    - special_event: Boolean indicating if there's a special event nearby

    Returns:
    - sales: Number of lemonade cups sold
    - profit: Net profit for the day
    - time_spent: Total time spent operating the stand
    """
    # Base demand model
    base_demand = 50 if weather == 'sunny' else 30 if weather == 'cloudy' else 10

    # Adjust demand for weekend, special events, and hours open
    if day == 'weekend':
        base_demand *= 1.5
    if special_event:
        base_demand *= 2
    base_demand *= (hours_open / 8)  # Assume max demand if open 8 hours

    # Influence of marketing spend (logarithmic growth)
    marketing_effectiveness = 10 * (1 - 1 / (1 + marketing_spend / 50))

    # Adjust demand for price (demand decreases as price increases, but not linearly)
    price_sensitivity = 50 - (price - 1) ** 2 if price < 5 else 20 - (price - 5)
    price_sensitivity = max(0, price_sensitivity)

    # Calculate final demand
    final_demand = base_demand + marketing_effectiveness + price_sensitivity

    # Calculate sales and profit
    sales = int(final_demand)
    cost_per_cup = 0.5 + lemon_cost  # Adjusted cost to include lemon cost
    profit = sales * (price - cost_per_cup) - marketing_spend

    # Time spent at the stand
    time_spent = hours_open

    return sales, profit, time_spent

Let run it once to see how it works:

In [ ]:
# Example simulation with uncertain lemon cost
weather = random.choice(['sunny', 'cloudy', 'rainy'])
day = random.choice(['weekday', 'weekend'])
price = round(random.uniform(1, 5), 2)
marketing_spend = random.randint(0, 100)
hours_open = random.randint(4, 8)  # Assume the stand can be open between 4 to 8 hours
lemon_cost = round(random.uniform(0.1, 0.5), 2)  # Uncertain purchase price of lemons
special_event = random.choice([True, False])

sales, profit, time_spent = simulate_lemonade_stand(weather, day, price, marketing_spend, hours_open, lemon_cost,
                                                    special_event)
print(
    f"Weather: {weather}, Day: {day}, Price: ${price}, Marketing Spend: ${marketing_spend}, Hours Open: {hours_open}, Lemon Cost: ${lemon_cost}, Special Event: {special_event}")
print(f"Sales: {sales} cups, Profit: ${profit:.2f}, Time Spent: {time_spent} hours")

## Part 1: Exploring the Scenario Space

Now that we've laid the foundation with our lemonade stand model it's time to dive deeper into the heart of strategic decision-making: exploring the scenario space. This phase is where creativity meets analysis. By imagining various possible futures and examining how different factors interact, we can uncover insights that guide our decisions and strategies.

### What you will learn

Here's a snapshot of what we'll cover:

- **Open Exploration**: We'll start by casting a wide net, considering a vast array of potential scenarios. This is about asking "What if?" in as many ways as we can imagine—what if it's a rainy weekend? What if a local event drives up demand? Through this exploratory process, you'll learn to identify which factors could significantly impact your lemonade stand's success.

- **Scenario Discovery**: Once we've considered a broad set of possibilities, we'll focus on identifying the most impactful scenarios using specialized techniques like the Patient Rule Induction Method (PRIM) and Subspace Partitioning. This step is crucial for narrowing down the scenarios to those that truly matter, enabling us to concentrate our efforts and resources more effectively.

- **Global Sensitivity Analysis**: Understanding which variables have the most significant influence on our outcomes is key. Through Global Sensitivity Analysis, we'll learn which factors (e.g., weather, pricing, marketing spend) you should focus on to improve your lemonade stand's performance. This analysis helps in prioritizing strategic adjustments and investments.

By exploring the scenario space, you'll gain invaluable skills in navigating uncertainties and making informed decisions. You'll learn not just to react to the business environment but to anticipate and prepare for various futures, making your lemonade stand—and any other venture you undertake—more resilient and successful.


### 1.1 Open Exploration
Open exploration involves systematically varying parameters of our lemonade stand model to observe how changes affect outcomes like sales and profit. It's akin to asking a series of "what if" questions:

- What if it rains all week?
- What if we double our marketing efforts on weekends?
- How does changing the lemon cost impact our profit?

- This approach not only helps us understand the dynamics of our lemonade stand but also teaches us valuable lessons about the impact of external and internal factors on any business.

#### Why open exploration?
Before we dive into specific techniques for scenario discovery and sensitivity analysis, we must first explore freely. This unbounded exploration allows us to:

- Identify a wide range of outcomes that our lemonade stand might face.
- Understand the impact of different variables in various combinations.
- Generate hypotheses about what might be most important for our success.

#### How to perform open exploration
To begin, we'll simulate a variety of scenarios by varying the inputs to our lemonade stand model. We'll change one factor at a time to observe its effect, and then combine multiple changing factors to see more complex interactions. This process will give us an initial sense of which variables have the most significant impact on our outcomes.

Let's start with a simple exploration, varying only the weather while keeping other factors constant.

In [ ]:
# Define a list of weather conditions
weather_conditions = ['sunny', 'cloudy', 'rainy']

# Set constants for other variables
day = 'weekend'
price = 2.50
marketing_spend = 20
hours_open = 8
lemon_cost = 0.30
special_event = False

# Simulate and print the outcomes for different weather conditions
for weather in weather_conditions:
    sales, profit, time_spent = simulate_lemonade_stand(weather, day, price, marketing_spend, hours_open, lemon_cost, special_event)
    print(f"Weather: {weather}, Sales: {sales}, Profit: ${profit:.2f}")

Next, we'll introduce variability in another dimension—say, day type—to observe how the interaction between weather and day type affects our results.

In [ ]:
# Define a list of day types and a list to collect simulation results
day_types = ['weekday', 'weekend']
results = []

# Run the simulation across different weather conditions and day types
for weather in weather_conditions:
    for day in day_types:
        sales, profit, _ = simulate_lemonade_stand(weather, day, price, marketing_spend, hours_open, lemon_cost, special_event)
        results.append({'Weather': weather, 'Day': day, 'Sales': sales, 'Profit': profit})

# Convert results into a DataFrame
df = pd.DataFrame(results)
df

**Visualizing the results**
Now that we have our simulation results in a DataFrame, let's create visualizations to better understand how weather conditions and day types affect sales and profits.

In [ ]:
# Setting up the figure and axes for the subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6)) # 1 row, 2 columns

# Titles and variables for each subplot
titles = ['Lemonade Sales: Weather Conditions vs. Day Type', 'Lemonade Stand Profit: Weather Conditions vs. Day Type']
variables = ['Sales', 'Profit']

for i, variable in enumerate(variables):
    sns.barplot(x='Weather', y=variable, hue='Day', data=df, palette='coolwarm', ax=axes[i])
    axes[i].set_title(titles[i])
    axes[i].set_xlabel('Weather Condition')
    axes[i].set_ylabel(variable)
    axes[i].legend(title='Day Type')

#### Try it yourself!

Now is a great moment to start understanding of the lemonade stand's dynamics through open exploration yourself. For example, try experimenting with these key variables on your own:

1. **Pricing Strategy**: Adjust the lemonade price ($1 to $5) and observe the impact on sales and profit. How do customers react to price changes on different weather days?
2. **Marketing Spend**: Experiment with varying your marketing budget ($0, $20, $50, $100) and see how it affects demand. Is marketing more effective on weekends or during special events?
3. **Operating Hours**: Change how long you're open (4 to 8 hours) and track the effect on profits. Is there a point where extra hours don't increase earnings?
4. **Lemon Cost Variability**: Simulate changes in lemon cost ($0.1 to $0.5 per lemon) to understand how input costs impact profitability. Can adjusting prices or marketing compensate for higher costs?

Consider simple charts to visualize the effects. This approach will help you grasp how different decisions can influence your lemonade stand's success, providing insights into effective strategy adjustments.

In [ ]:
# Do some open exploration here!


Let's take a moment to reflect on our graphs. They show us that sunny weekends are great for lemonade sales and profits, but there's more to the story. These snapshots don't consider other important factors like lemon prices or whether there's a big game in town, which could shake things up.

In the upcoming sections, we'll dig deeper. We'll uncover which combinations of factors, like a heatwave or a soccer tournament, really boost our sales. And we'll get savvy with our strategy, adjusting prices and planning special offers to make the most of every sunny day—or any day, for that matter. Stay tuned, as we get ready to add layers and learn how to make our lemonade stand the talk of the town, no matter the weather!

### 1.2 Scenario Discovery

After embarking on the journey of open exploration, where we cast a wide net to understand the myriad ways in which different factors impact our lemonade stand, we move on to a more focused and powerful phase: scenario discovery. This phase is where we wield sophisticated tools to sift through the vast data we've generated, pinpointing the scenarios that matter most to our success. The spotlight here is on two powerful techniques: the Patient Rule Induction Method (PRIM) and Subspace Partitioning.

#### The Power of Scenario Discovery

Scenario discovery allows us to go beyond simple observations, helping us identify patterns and key scenarios that could significantly impact our lemonade stand's outcomes. By applying PRIM and Subspace Partitioning, we can uncover specific combinations of conditions that lead to particularly high or low sales and profits. This focused approach enables us to prepare better and optimize our strategies for these critical scenarios.

#### Why Scenario Discovery?

- **Focus**: While open exploration spreads our attention across all possibilities, scenario discovery helps us focus on what truly matters, saving us time and resources.
- **Insight**: It provides deep insights into the combinations of factors that drive success or failure, offering clarity in complex decision-making landscapes.
- **Strategy Optimization**: By understanding key scenarios, we can tailor our strategies to exploit opportunities or mitigate risks, enhancing our lemonade stand's performance.

#### How to Perform Scenario Discovery

##### Patient Rule Induction Method (PRIM)

PRIM helps us identify "boxes" in our multidimensional dataset where the outcome (e.g., high sales) is significantly more likely. It's akin to finding pockets of gold in a vast mine by systematically narrowing down where to dig based on the evidence.

1. **Setup**: Begin with your dataset from open exploration, including all scenarios and their outcomes.
2. **Application**: Apply PRIM to search for areas within your dataset where, for example, profits are highest. PRIM will iteratively search for conditions that lead to these high-profit scenarios, such as a combination of sunny weather, weekend days, and specific pricing strategies.

##### Subspace Partitioning

Subspace Partitioning takes a different approach, dividing the dataset into subspaces based on the variables (e.g., weather, day type) and then analyzing each subspace to identify where outcomes like high sales occur more frequently.

1. **Divide**: Segment your data based on key variables, creating subspaces for analysis.
2. **Analyze**: For each subspace, look for patterns or conditions that consistently lead to desired outcomes.